# Exploratory data analysis (EDA)

Approaching EDA systematically is important to ensure that you don't forget any steps and avoid making errors which could compromise your analysis/modeling later.  Before starting EDA, you should have defined the business question and understand what data sources are available.  During EDA, you want to refine your intuition about the data and develop hypotheses which we could model or test later.

Note how at each step, I keep checking that everything worked as expected.

We will perform the following steps:

*  Load data
*  Sanity check data
*  Summary statistics
*  Plots
*  Identify and examine problem data
   *  Outliers
   *  Missing data
*  Clean data
*  First crappy model

After EDA, it is time to peform cleaning for more intensive modeling, engineer features, and refine the model.

In [ ]:
%matplotlib inline

## Load data

Everything starts with loading data.

In [ ]:
df = df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', 
                      names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])

## Sanity checking

After loading the data, you need to check that the data loaded correctly.  If you are working with a large dataset, create a subset so that you can quickly get everything working.  Later, rerun with the full data or larger subsamples.  Make sure that the subsamples are representative by trying multiple subsamples.

In [ ]:
df.head()

In [ ]:
df.tail()

Check that data types are correct and if there any data is missing:

In [ ]:
df.info()

Convert the `species` to a categorical variable: 

In [ ]:
df.species = df.species.astype('category')

Check conversion was successful:

In [ ]:
df.info()

Great!  Everything is now the correct data type.  And, we are fortunate not to have any missing values.

I like to tabulate (1-way and 2-way) categorical variables:

In [ ]:
df.species.value_counts()

Looks like the data is ready for the next step: summary statistics.  Note:  there is no class imbalance.  If there were, we would have to think about up/down sampling or weighting.

## Summary statistics

Next, I look at summary statistics. I want to understand the nature of the variation and whether there are any problems, such as outliers or missing values.  You may decide to plot at this stage as well -- histograms and box plots are particularly useful.  You may also decide to compute statistics for subsets of the data based on some categorical field.

Let's start with the classic statistics:

In [ ]:
df.describe().T

If the data is very skewed or has kurtosis, you might want to check other percentiles or statistics.  Or, check for departures from normality.  Big data, such as Amazon data, often has a long tail, so you may need to transform data during feature engineering, such as by taking `log`s of fields.

Check that statistics look different for each species so that we have a chance of predicting species:

In [ ]:
df.groupby('species').describe()

## Plots

Summary statistics can be misleading, as Anscomb showed with his [Quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet).  Consequently, you should always plot (a subset) of your data to develop intuition and hypotheses about the data.

I like to start by plotting the features against each other to better understand the variation in the data.  If you have a lot of features, you will need to chose the sensible subset.

In [ ]:
from pandas.tools.plotting import scatter_matrix
scatter_matrix(df, alpha=0.3, c=df.species.cat.codes, figsize=(16,16))

These plots make us hopeful that the variation in the data will be able to predict the different species.  I like to start with box plots which graphically show the mean, median, quartiles, and outliers.

In [ ]:
df.plot(kind='box', figsize=(9,9))

Only `sepal_width` has outliers, but they are not extremely extreme.

In [ ]:
df.plot(bins=25, alpha=0.2, kind='hist')

The data seems to have good variation, but it looks like there might be a mass point around `0` for `petal_width`.  Let's look into that in more detail:

In [ ]:
df.petal_width.plot(bins=25, alpha=0.3, kind='hist')

We might need to talk to a domain expert (biologist) about this. Or, it could be different species.

In [ ]:
df.groupby('species').hist('petal_width', bins=10)

The split in the histogram of `petal_width` appears to be caused by the iris's species.

## Examine bad data

It is important to identify outliers and missing values, and then make a sensible decision about how to deal with them.

### Outliers

We saw from the box plots that only `sepal_width` has outliers.  Some options are:

*  Drop them
*  Use them unchanged
*  Bin the continuous feature and add a bin for outliers (Check out `pd.cut` and `pd.get_dummies` for this case)
*  [Winsorize](https://en.wikipedia.org/wiki/Winsorising) the outliers
*  Truncate or impute the outliers in some other sensible manner
*  Check if some other factor is causing outliers, such as defects in telemetry/instrumentation

In the current case, using them unchanges is reasonable because the outliers are not very extreme.

### Missing data

Similarly, you need to identify missing values and decide how to deal with them.  There are a couple options:

*  Drop rows with missing data if it is *missing at random* (How would you test this?)
*  Impute missing values
*  Bin the feature and add a bin for missing

## Clean data

Be very careful when cleaning data not to lose information which has predictive power.  It is very helpful to discuss cleaning and feature engineering with a domain expert.  Fortunately, this data doesn't need any further cleaning.  Often, you will have to identify and construct a label/target which makes sense and engineer other features.

## Build a crappy first model

At this step, you want to check that your pipeline works and modeling is likely to succeed.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
X = df.icol(range(4))
y = df.icol(4)
y = y.cat.codes

Sanity check:

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
model = LogisticRegressionCV(cv=3)

In [ ]:
model.fit(X,y)

In [ ]:
y_hat = model.predict(X)

In [ ]:
model.score(X,y)

Be suspicious of results which are too good.  If you observe them, you need to check for **information leakage**.

In [ ]:
df[y != y_hat]

## Final remarks

EDA, cleaning, feature engineering, and modeling are an interative process.  At each step, you might discover something you missed and need to return to an earlier step.  Ultimately, understanding your data's strengths and weaknesses and engineering good features is usually more important that what model you choose.

Now, you should focu